# TF-IDF e busca por conteúdo

Nesta atividade, vamos lidar com a seguinte situação: temos um grande banco de dados com textos, e queremos encontrar qual texto é mais relevante para uma consulta. Esse problema aparece em buscadores como Google, e também em sistemas locais como ElasticSearch.

In [2]:
import pandas as pd

DATASET = 'datasets/wikipedia_movies.zip'
df = pd.read_csv(DATASET).sample(1000)
df = df[['Title', 'Plot']]
print(df.head(), len(df))

                           Title  \
17961              Banglar Nayok   
28811             Badmash Gujjar   
6641           Runaway Daughters   
17191                    Get Out   
7970   The Flight of the Phoenix   

                                                    Plot  
17961  Press reporter old man 'Anowar Hossain' (Anowa...  
28811  Shaan is a charming, rustic, God-fearing local...  
6641   Audrey Barton is the teen daughter of rich, ir...  
17191  African-American photographer Chris Washington...  
7970   Frank Towns (James Stewart) is the pilot of a ...   1000


## Exercício 1
**Objetivo: lembrar-se do que é TF e o que é DF**

Identifique o Term Frequency e o Document Frequency nas asserções abaixo:

1. Quanto maior o ___, mais comum é a palavra entre os documentos de uma coleção
1. Quanto maior o ___, mais vezes a palavra é mencionada num documento específico
1. $P(w | \text{documento})$
1. $P(w | \text{coleção})$
1. Ajuda a identificar a coleção da qual um documento faz parte
1. Ajuda a identificar um documento dentro de uma coleção

1: DF
2: DF
3: TF
4: DF
5: DF
6: TF

## Exercício 2
**Objetivo: refletir sobre o uso de TF-IDF**

A medida TFIDF diz o quão relevante um documento é dentro de uma coleção e em relação a uma palavra específica. Ela é calculada para um par palavra-documento como:

$\text{TFIDF = TF / DF}$

Quando um documento tem um TFIDF alto em relação a uma palavra, isso significa que:

1. A palavra tende a ser (comum / incomum)
1. O documento menciona a palavras (muitas / poucas) vezes

Portanto, qual seria uma maneira de escrever um documento que tem intencionalmente um TFIDF alto para uma palavra?

## Exercício 3
**Objetivo: calcular TFIDF para documentos usando sklearn**

TFIDF pode ser entendido como um processo de vetorização, semelhante a usar o CountVectorizer. Abaixo, há um código que mostra um exemplo dessa vetorização usando sklearn. 

1. Escolhendo um filme aleatório da coleção que carregamos, identifique o TFIDF das palavras "zombie", "fungus" e "survival".
1. Identifique o filme que tem o maior TFIDF para a palavra "zombie".

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(max_features=10000)
tfidf = vectorizer.fit_transform(df['Plot'])
# print(vectorizer.vocabulary_)


In [4]:
n_filme = 0
j = vectorizer.vocabulary_['assistant']
print(df.iloc[n_filme]['Title'], tfidf[n_filme, j])

Banglar Nayok 0.0


In [5]:
TERMO_DE_BUSCA = "death"
j =  vectorizer.vocabulary_[TERMO_DE_BUSCA]
m = 0
m_max = 0
m_i = 0
for n_filme in range(tfidf.shape[0]):
    m = tfidf[n_filme, j]
    if m>m_max:
        m_max = m
        m_i = n_filme

print(df.iloc[m_i]['Title'], tfidf[m_i, j])
print(df.iloc[m_i]['Plot'])

Downstream 0.22901642371244585
A detective goes undercover by taking a job as a bargee in order to prevent a woman's death.[2]


In [6]:
TERMO_DE_BUSCA = "death"
j =  vectorizer.vocabulary_[TERMO_DE_BUSCA]
local_tfidf = tfidf[:,j]
m_i = np.argmax(local_tfidf)
m_max = local_tfidf[m_i]

print(df.iloc[m_i]['Title'], tfidf[m_i, j])
print(df.iloc[m_i]['Plot'])

Downstream 0.22901642371244585
A detective goes undercover by taking a job as a bargee in order to prevent a woman's death.[2]


## Exercício 4
**Objetivo: implementar uma busca por vários termos simultaneamente**

Uma possível maneira de implementar uma busca por vários termos é somar o TFIDF de todas as palavras da query para cada documento da coleção, e então retornar o documento que tem a maior soma. Por exemplo, numa busca por "zombie fungus survival" deveríamos somar, para cada documento, o TFIDF de "zombie", de "fungus" e de "survival" e então ordenar o resultado.

1. Escreva código que implemente uma busca na base de dados de filmes à partir de uma query específica.
1. Qual é a complexidade ($O(...)$) da sua busca?

In [7]:
import re
query = "zombie fungus survival"

palavras = re.findall("\w+", query)
# print(palavras)
# Implemente sua solução aqui
j = [vectorizer.vocabulary_[t] for t in palavras if t in vectorizer.vocabulary_.keys()]
local_tfidf = tfidf[:,j].todense()
local_tfidf = np.min(local_tfidf, axis=1)

m_i = np.argmax(local_tfidf)
m_max = local_tfidf[m_i]

print(df.iloc[m_i]['Title'], tfidf[m_i, j])
print("----")
print(df.iloc[m_i]['Plot'])

Banglar Nayok 
----
Press reporter old man 'Anowar Hossain' (Anowar Hossain) published the daily news paper an most wanted criminal 'Rowshan Chowdhury’s' (Ahmed Sharif) criminal effect. After someday Rowshan Chowdhury killed Press reporter Anowar Hossain and his adolescent 'Rokeya' (Shabana) promises with him these killers take most punishment venture. After 12 year Rokeya gets be a professional lawyer. And a competitor mafia 'Don' (Jashim) he can trust to arms is all power with encouragement, also Rokeya with encouragement can trust to pen is all power. Rokeya’s small brother 'Munna' (Riaz) student of a university and 'Jully' (Sonia) also same university student. First time botheration with these duo and second time through the love relation. But Jully’s brother accepts it not.Rowshan Chowdhury and Don now start a war for take Don place.Don's sister Jully has gone to her lover Munna.Don losts his Don place to save his sister.Don now not Don he is now Banglar Nayok. 'Banglar Nayok' at 

In [8]:
j =  vectorizer.vocabulary_[TERMO_DE_BUSCA]
local_tfidf = tfidf[:,j]

m_i = np.argmax(local_tfidf)
m_max = local_tfidf[m_i]

print(df.iloc[m_i]['Title'], tfidf[m_i, j])
print(df.iloc[m_i]['Plot'])

Downstream 0.22901642371244585
A detective goes undercover by taking a job as a bargee in order to prevent a woman's death.[2]


## Exercício 5
**Objetivo: implementar um índice invertido**

Você provavelmente reparou (talvez não tenha reparado, e é tudo bem) que, para fazer a busca, até agora, teve que varrer todos os documentos da sua coleção. Isso provavelmente levaria algum tempo, especialmente quando a coleção começa a aumentar. Para evitar ter que varrer todos os documentos da coleção, podemos implementar uma técnica chamada *índice invertido*. A ideia do índice invertido é usar um dicionário cujas chaves são as palavras do vocabulário e cujo conteúdo é uma lista de documentos que contém essa palavra, possivelmente acompanhados do TFIDF correspondente. Por exemplo:

In [9]:
indice = { 'palavra_1' : {'documento_1': 0.5, 'documento_2': 0.1}, 
          'palavra_2' : {'documento_2': 0.6},
            'equalization' : {'documento_3': 0.7}    }

def buscar(palavras, indice):
    assert type(palavras)==list
    resultado = dict()
    for p in palavras:
        if p in indice.keys():
            for documento in indice[p].keys():
                if documento not in resultado.keys():
                    resultado[documento] = indice[p][documento]
                else:
                    resultado[documento] += indice[p][documento]
    return resultado

buscar(['palavra_1', 'palavra_2', 'equalization'], indice)#retornando a relevancia de documentos para a query

{'documento_1': 0.5, 'documento_2': 0.7, 'documento_3': 0.7}

1. Adicione uma nova palavra ao índice e escolha seu TFIDF. Realize uma nova busca e verifique o resultado.
1. Escreva uma função que ordena o resultado e retorna apenas `N` documentos mais relevantes para sua busca.
1. Incremente sua biblioteca de forma que ela passe a receber uma string como entrada (representando a query) e retorne os `N` documentos mais relevantes (`N` pode ser definido arbitrariamente).

In [13]:
# versao original alterada para ter funcao de encontrar n_maiores
indice = { 'palavra_1' : {'documento_1': 0.5, 'documento_2': 0.1}, 
          'palavra_2' : {'documento_2': 0.6}    }

def buscar(palavras, indice):
    assert type(palavras)==list
    resultado = dict()
    for p in palavras:
        if p in indice.keys():
            for documento in indice[p].keys():
                if documento not in resultado.keys():
                    resultado[documento] = indice[p][documento]
                else:
                    resultado[documento] += indice[p][documento]
    return resultado

res = buscar(['palavra_1', 'palavra_2'], indice)#retornando a relevancia de documentos para a query

def n_maiores(res_busca, n):
    res = []
    for k in res_busca.keys():
        res.append((res_busca[k],k))
    res = sorted(res, reverse = True)[0:n]
    return res

print(n_maiores(res, 2))

def query(query_string, n, indice):
    palavras = re.findall('\w+', query_string)
    res = buscar(palavras, indice)
    res_n = n_maiores(res, n)
    return res_n

print(query("palavra_1 palavra_2 equalization", 2, indice))

[(0.7, 'documento_2'), (0.5, 'documento_1')]
[(0.7, 'documento_2'), (0.5, 'documento_1')]


## Exercício 6
**Objetivo: implementar um buscador de filmes**

Implemente uma função que recebe como entrada uma query e retorna os títulos e enredos dos 5 filmes mais relevantes para aquela query. Se precisar, use mais parâmetros ou variáveis globais. Teste a sua função e veja se você concorda com os resultados, incluindo se você consegue encontrar seus filmes favoritos e se consegue alguma recomendação relevante a um filme novo.

In [11]:
from tqdm import tqdm
indice_filmes = dict()

for w in tqdm(vectorizer.vocabulary_.keys()):
    indice_filmes[w] = dict()
    for j in range(tfidf.shape[0]):
        if(tfidf[j, vectorizer.vocabulary_[w]])>0:
            indice_filmes[w][j] = tfidf[j, vectorizer.vocabulary_[w]]
def query_movies(query : str):
    pass

100%|██████████| 10000/10000 [01:13<00:00, 135.78it/s]


In [14]:
res = query("king", 3, indice_filmes)
for n in res:
    print(df.iloc[n[1]]['Title'])
    print(df.iloc[n[1]]['Plot'])
    print("--")

The Madness of King George
The film depicts the ordeal of King George III whose bout of madness in 1788 touched off the Regency Crisis of 1788, triggering a power struggle between factions of Parliament under the conservative William Pitt the Younger and the reform-minded Charles James Fox.
At first, the King's habits appear mildly eccentric, and are purposely ignored for reasons of state. The King is seen as being highly concerned with the wellbeing and productivity of England, and continually exhibits an encyclopedic knowledge of the families of even the most obscure royal appointments. In fact, the King is growing more unsettled, largely over the loss of America. George, his oldest son, aggravates the situation, knowing that he would be named regent in the event the King was found incapacitated. George chafes under his father's repeated criticism, but also hopes for regency to allow him greater freedom to marry his Catholic mistress. George also knows that he has the moral support o

## Exercício 7
**Objetivo: identificar palavras-chave usando TFIDF**

Uma outra aplicação de TFIDF é encontrar palavras-chave, isto é, palavras que diferenciam um documento do restante dos documentos de sua coleção.

Incremente seu buscador de forma que, além do título e enredo, ele também escolha as algumas palavras (escolha quantas!) mais relevantes de cada documento e as imprima como keywords.

## Exercício 8
**Objetivo: encontrar documentos semelhantes usando TFIDF**

Uma maneira de encontrar documentos semelhantes em uma coleção de textos é assumir que o texto do documento é uma query, e então realizar a busca normalmente. O problema disso é que provavelmente teríamos textos muito longos e a query ficaria muito carregada. Para solucionar isso, poderíamos usar apenas as palavras mais relevantes de um documento como query. Implemente uma função que recebe o índice (ou outro identificador único) de um documento de nosso banco de dados e então encontra 5 documentos semelhantes a ele.